In [1]:
#import dependencies
import pandas as pd
import matplotlib as plt
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
import sklearn as skl
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from collections import Counter
from imblearn.datasets import make_imbalance
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline
from imblearn.metrics import classification_report_imbalanced

In [2]:
#read in the data
#take input from provisional database (csv)
#df = pd.read_csv(Path('./Resources/all_responses_coded.csv'))
# Load the csv file from GitHub
url = 'https://raw.githubusercontent.com/mandymccabe/Final_Project/janet_branch/Data/Final_Project_Full.csv'
url2= 'https://raw.githubusercontent.com/mandymccabe/Final_Project/main/Resources/all_responses_coded.csv'
df = pd.read_csv(url, index_col=0)
df2 = pd.read_csv(url2, index_col=0)
df.head(10)


,political_views,trust_in_press,online_paid_access,impression_with_access_to_data,trump_as_president,age,gender,income,us_region,startdate,...,usa_today,washington_post,fox_news,breitbart,cnn,buzzfeed,huffington_post,time,us_news_and_world_report,other
respondentid,,,,,,,,,,,,,,,,,,,,,
6176264298,Moderate,Fair Amount,No,No Change,Strongly Disapprove,30-44,Male,"$10,000-$24,999",East South Central,2017-05-01,...,1,0,0,0,1,0,0,1,0,0
6176263960,Moderate,Fair Amount,No,Increase Trust,Somewhat Disapprove,18-29,Female,"$0-$9,999",Middle Atlantic,2017-05-01,...,0,0,0,0,1,0,0,0,0,0
6176258621,Liberal,Fair Amount,Yes,Increase Trust,Strongly Disapprove,30-44,Male,"$125,000-$149,999",East North Central,2017-05-01,...,0,1,0,0,0,0,0,0,0,BBC
6176257082,Liberal,Fair Amount,No,Increase Trust,Strongly Disapprove,18-29,Male,"$125,000-$149,999",Mountain,2017-05-01,...,0,1,0,0,0,0,0,0,0,"BBC News, and local news outlets"
6176256111,Liberal,Fair Amount,Yes,No Change,Strongly Disapprove,30-44,Male,"$10,000-$24,999",Pacific,2017-05-01,...,1,0,0,0,1,1,0,1,0,0
6176254380,Liberal,Not Very Much,No,Increase Trust,Strongly Disapprove,30-44,Female,"$200,000 & up",Mountain,2017-05-01,...,0,1,0,0,0,0,0,0,0,0
6176253761,Conservative,Not Very Much,No,Increase Trust,Strongly Approve,30-44,Male,"$50,000-$74,999",East North Central,2017-05-01,...,0,0,0,0,0,0,0,0,0,None
6176248227,Moderate,Not Very Much,No,Increase Trust,Somewhat Disapprove,18-29,Male,"$10,000-$24,999",Pacific,2017-05-01,...,0,0,1,0,1,0,0,0,0,0
6176247712,Moderate,Fair Amount,No,Increase Trust,Strongly Disapprove,30-44,Male,"$25,000-$49,999",West North Central,2017-05-01,...,0,0,0,0,0,0,0,0,0,"NPR, Reuters, New Yorker, Al Jazeera English, ..."


In [3]:
df2.head()

,RespondentID,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A55,A56,A57,A58,A59,A60,A61,A62,StartDate,EndDate
index,,,,,,,,,,,,,,,,,,,,,
0,6176264298,0,0,1,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,5/1/17 15:41,5/1/17 15:43
1,6176263960,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,5/1/17 15:35,5/1/17 15:43
2,6176258621,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,5/1/17 15:38,5/1/17 15:40
3,6176257082,0,0,0,1,0,0,1,0,0,...,1,0,1,0,0,0,0,0,5/1/17 15:38,5/1/17 15:39
4,6176256111,0,0,0,1,0,0,1,0,0,...,0,1,1,0,0,0,0,0,5/1/17 15:34,5/1/17 15:39


In [4]:
PoliticalViews = df.filter(["political_views"], axis=1)
PoliticalViews.head()

,political_views
respondentid,
6176264298,Moderate
6176263960,Moderate
6176258621,Liberal
6176257082,Liberal
6176256111,Liberal


In [5]:
#drop columns we don't want to use to predict and those that do not have an int value

df2 = df2.drop(["A21", "StartDate", "EndDate"], axis=1)
df2.head()

,RespondentID,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A53,A54,A55,A56,A57,A58,A59,A60,A61,A62
index,,,,,,,,,,,,,,,,,,,,,
0,6176264298,0,0,1,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0
1,6176263960,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,6176258621,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,6176257082,0,0,0,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
4,6176256111,0,0,0,1,0,0,1,0,0,...,0,0,0,1,1,0,0,0,0,0


In [6]:
merged_dfs = pd.merge(df2, PoliticalViews, how='outer', left_on=["RespondentID"], right_on=['respondentid'])
#merged_df2 = merged_dfs.drop(['political_views_x'], axis=1)
merged_dfs.head()

,RespondentID,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A54,A55,A56,A57,A58,A59,A60,A61,A62,political_views
0,6176264298,0,0,1,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,Moderate
1,6176263960,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,Moderate
2,6176258621,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,Liberal
3,6176257082,0,0,0,1,0,0,1,0,0,...,0,1,0,1,0,0,0,0,0,Liberal
4,6176256111,0,0,0,1,0,0,1,0,0,...,0,0,1,1,0,0,0,0,0,Liberal


In [7]:
#merged_df2 = merged_df2.drop(["RespondentID"], axis=1)
#merged_df2.head()

In [8]:
#set target and x values
X=merged_dfs[list(merged_dfs.columns)[6:-1]]
X.head()

,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,...,A53,A54,A55,A56,A57,A58,A59,A60,A61,A62
0,0,1,0,0,1,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
1,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,1,0,0,0,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
4,0,1,0,0,1,1,1,0,0,0,...,0,0,0,1,1,0,0,0,0,0


In [9]:
# Create the StandardScaler instance
#scaler = StandardScaler()

# Fit the StandardScaler
#X_scaler = scaler.fit(X_train)

# Scale the data
#X_train_scaled = X_scaler.transform(X_train)
#X_test_scaled = X_scaler.transform(X_test)

In [10]:
#Y
y=merged_dfs[list(merged_dfs.columns)[-1]]
y.head(10)

0             Moderate
1             Moderate
2              Liberal
3              Liberal
4              Liberal
5              Liberal
6         Conservative
7             Moderate
8             Moderate
9    Very Conservative
Name: political_views, dtype: object

In [11]:
#split into training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42)

In [12]:
print('Training target statistics: {}'.format(Counter(y_train)))
print('Testing target statistics: {}'.format(Counter(y_test)))

Training target statistics: Counter({'Moderate': 313, 'Liberal': 161, 'Conservative': 161, 'Very Liberal': 85, 'Very Conservative': 45})
Testing target statistics: Counter({'Moderate': 107, 'Liberal': 59, 'Conservative': 54, 'Very Liberal': 26, 'Very Conservative': 10})


Decision Tree

In [13]:
DTC=DecisionTreeClassifier(random_state=42)
DTC = DTC.fit(X_train, y_train)

In [14]:
predictions = DTC.predict(X_test)

In [15]:
#print("Decision Tree Train Accuracy:", metrics.accuracy_score(y_train, DTC.predict(X_train)))
#print("Decision Tree Test Accuracy:", metrics.accuracy_score(y_test, DTC.predict(X_test)))

In [16]:
y_pred= DTC.predict(X_test)
print('Accuracy of decision tree classifier on test set: {:.2f}'.format(DTC.score(X_test, y_test)))

Accuracy of decision tree classifier on test set: 0.35


In [17]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

                   precision    recall  f1-score   support

     Conservative       0.32      0.30      0.31        54
          Liberal       0.29      0.25      0.27        59
         Moderate       0.48      0.48      0.48       107
Very Conservative       0.00      0.00      0.00        10
     Very Liberal       0.20      0.27      0.23        26

         accuracy                           0.35       256
        macro avg       0.26      0.26      0.26       256
     weighted avg       0.35      0.35      0.35       256



#accuracy is very poor for 5 outputs, will combine very conservative/conservative and very liberal/liberal respondents to condense to 3 output options

In [18]:
PoliticalViews3 = PoliticalViews.replace(regex={r'Very Conservative': 'Conservative', 'Very Liberal': 'Liberal'})
PoliticalViews3.head(10)

,political_views
respondentid,
6176264298,Moderate
6176263960,Moderate
6176258621,Liberal
6176257082,Liberal
6176256111,Liberal
6176254380,Liberal
6176253761,Conservative
6176248227,Moderate
6176247712,Moderate


In [19]:
PoliticalnumViews = PoliticalViews3.replace(regex={r'Conservative': '1', 'Moderate': '2', 'Liberal':'3'})
PoliticalnumViews.head()

,political_views
respondentid,
6176264298,2
6176263960,2
6176258621,3
6176257082,3
6176256111,3


In [20]:
#create dummy variables for multiclass predictor
PoliticalnumViews = pd.get_dummies(PoliticalnumViews["political_views"])
#PoliticalViews = PoliticalViews.drop(["Very 3"], axis=1)
PoliticalnumViews.head(10)

,1,2,3
respondentid,,,
6176264298,0,1,0
6176263960,0,1,0
6176258621,0,0,1
6176257082,0,0,1
6176256111,0,0,1
6176254380,0,0,1
6176253761,1,0,0
6176248227,0,1,0
6176247712,0,1,0


In [21]:
df3 = df2.drop(["A1", "A2", "A3", "A4", "A5"], axis=1)
df3.head()


,RespondentID,A6,A7,A8,A9,A10,A11,A12,A13,A14,...,A53,A54,A55,A56,A57,A58,A59,A60,A61,A62
index,,,,,,,,,,,,,,,,,,,,,
0,6176264298,0,1,0,0,1,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0
1,6176263960,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,6176258621,0,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,6176257082,0,1,0,0,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
4,6176256111,0,1,0,0,1,1,1,0,0,...,0,0,0,1,1,0,0,0,0,0


In [22]:
df3 = pd.merge(df3, PoliticalnumViews, how='outer', left_on=["RespondentID"], right_on=['respondentid'])
df3 = pd.merge(df3, PoliticalViews3, how='outer', left_on=["RespondentID"], right_on=['respondentid'])
df3 = df3.drop(['RespondentID'], axis=1)
df3.head(10)

,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,...,A57,A58,A59,A60,A61,A62,1,2,3,political_views
0,0,1,0,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,1,0,Moderate
1,0,1,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,Moderate
2,0,1,0,0,1,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,Liberal
3,0,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,1,Liberal
4,0,1,0,0,1,1,1,0,0,0,...,1,0,0,0,0,0,0,0,1,Liberal
5,0,0,1,0,1,1,0,1,0,0,...,0,0,0,1,0,0,0,0,1,Liberal
6,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,Conservative
7,0,0,1,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,Moderate
8,0,1,0,0,1,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,Moderate
9,0,0,0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,Conservative


In [23]:
# set target data  and x values
X3=df3[list(df3.columns)[:-4]]
print(X.shape)
X3.head()

(1021, 56)


,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,...,A53,A54,A55,A56,A57,A58,A59,A60,A61,A62
0,0,1,0,0,1,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
1,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,1,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,1,0,0,0,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
4,0,1,0,0,1,1,1,0,0,0,...,0,0,0,1,1,0,0,0,0,0


In [24]:
#Y
y3=df3[list(df3.columns)[-1]]
y3.head(10)

0        Moderate
1        Moderate
2         Liberal
3         Liberal
4         Liberal
5         Liberal
6    Conservative
7        Moderate
8        Moderate
9    Conservative
Name: political_views, dtype: object

In [25]:
#split into training and testing
X3_train, X3_test, y3_train, y3_test = train_test_split(
    X3, y3, test_size=0.30)

In [26]:
# summarize class distribution
print('Training target statistics with 3 outputs: {}'.format(Counter(y3_train)))
print('Testing target statistics with 3 outputs: {}'.format(Counter(y3_test)))

Training target statistics with 3 outputs: Counter({'Moderate': 295, 'Liberal': 221, 'Conservative': 198})
Testing target statistics with 3 outputs: Counter({'Moderate': 125, 'Liberal': 110, 'Conservative': 72})


In [27]:
#Decision Tree for set of 3
DTC3 = DecisionTreeClassifier(random_state=42)
DTC3 = DTC3.fit(X3_train, y3_train)

In [28]:
predictions3 = DTC3.predict(X3_test)

In [29]:
y3_pred= DTC3.predict(X3_test)
print('Accuracy of decision tree classifier on test set: {:.2f}'.format(DTC3.score(X3_test, y3_test)))

Accuracy of decision tree classifier on test set: 0.45


In [30]:
print(classification_report(y3_test, y3_pred))

              precision    recall  f1-score   support

Conservative       0.49      0.50      0.50        72
     Liberal       0.50      0.47      0.49       110
    Moderate       0.38      0.40      0.39       125

    accuracy                           0.45       307
   macro avg       0.46      0.46      0.46       307
weighted avg       0.45      0.45      0.45       307



Undersampling

In [31]:
#split into training and testing
X3_train_under, X3_test_under, y3_train_under, y3_test_under = train_test_split(
    X3, y3, test_size=0.30)

In [32]:
# summarize class distribution
print("Before undersampling: ", Counter(y3_train_under))

# define undersampling strategy
undersample = RandomUnderSampler(sampling_strategy='majority')

# fit and apply the transform
X3_train_under, y3_train_under = undersample.fit_resample(X3_train_under, y3_train_under)

# summarize class distribution
print("After undersampling: ", Counter(y3_train_under))

Before undersampling:  Counter({'Moderate': 297, 'Liberal': 244, 'Conservative': 173})
After undersampling:  Counter({'Liberal': 244, 'Conservative': 173, 'Moderate': 173})


In [33]:
#Decision Tree for set of 3 undersampled
DTC3_under = DecisionTreeClassifier(random_state=42)
DTC3_under = DTC3.fit(X3_train_under, y3_train_under)

In [34]:
predictions3_under = DTC3.predict(X3_test_under)

In [35]:
y3_pred_under= DTC3.predict(X3_test_under)
print('Accuracy of decision tree classifier on test undersampled: {:.2f}'.format(DTC3.score(X3_test_under, y3_test_under)))

Accuracy of decision tree classifier on test undersampled: 0.52


In [36]:
print(classification_report(y3_test_under, y3_pred_under))

              precision    recall  f1-score   support

Conservative       0.67      0.61      0.64        97
     Liberal       0.44      0.61      0.51        87
    Moderate       0.48      0.39      0.43       123

    accuracy                           0.52       307
   macro avg       0.53      0.54      0.53       307
weighted avg       0.53      0.52      0.52       307



Over Sampling

In [38]:
#split into training and testing
X3_train_over, X3_test_over, y3_train_over, y3_test_over = train_test_split(
    X3, y3, test_size=0.30)

In [40]:

# import SMOTE oversampling and other necessary libraries 
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# summarize class distribution
print("Before oversampling: ",Counter(y3_train_over))

# define oversampling strategy
SMOTE = SMOTE()

# fit and apply the transform
X3_train_SMOTE, y3_train_SMOTE = SMOTE.fit_resample(X3_train_over, y3_train_over)

# summarize class distribution
print("After oversampling: ",Counter(y3_train_SMOTE))

Before oversampling:  Counter({'Moderate': 294, 'Liberal': 242, 'Conservative': 178})
After oversampling:  Counter({'Conservative': 294, 'Liberal': 294, 'Moderate': 294})


In [41]:
#Decision Tree for set of 3 undersampled
DTC3_under = DecisionTreeClassifier(random_state=42)
DTC3_under = DTC3.fit(X3_train_SMOTE, y3_train_SMOTE)

In [42]:
predictions3_over = DTC3.predict(X3_test_over)

In [43]:
y3_pred_over= DTC3.predict(X3_test_over)
print('Accuracy of decision tree classifier on test oversampled: {:.2f}'.format(DTC3.score(X3_test_over, y3_test_over)))

Accuracy of decision tree classifier on test oversampled: 0.52


In [45]:
print(classification_report(y3_test_over, y3_pred_over))

              precision    recall  f1-score   support

Conservative       0.61      0.62      0.61        92
     Liberal       0.48      0.61      0.53        89
    Moderate       0.48      0.38      0.42       126

    accuracy                           0.52       307
   macro avg       0.52      0.54      0.52       307
weighted avg       0.52      0.52      0.51       307



Combination over and undersampling



In [46]:
#split into training and testing
X3_train_over_under, X3_test_over_under, y3_train_over_under, y3_test_over_under = train_test_split(
    X3, y3, test_size=0.30)

In [47]:
# import sampling and other necessary libraries 
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

ROC AUC score for the combined sampling method: nan


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/imblearn/pipeline.py", line 262, in fit
    Xt, yt = self._fit(X, y, **fit_params_steps)
  File "/usr/local/lib/python3.7/dist-packages/imblearn/pipeline.py", line 226, in _fit
    **fit_params_steps[name],
  File "/usr/local/lib/python3.7/dist-packages/joblib/memory.py", l

In [ ]:
# values to evaluate
over_values = [0.3,0.4,0.5]
under_values = [0.7,0.6,0.5]
for o in over_values:
  for u in under_values:
    # define pipeline
    model = SVC()
    over = SMOTE(sampling_strategy=o)
    under = RandomUnderSampler(sampling_strategy=u)
    steps = [('over', over), ('under', under), ('model', model)]
    pipeline = Pipeline(steps=steps)
    # evaluate pipeline
    scores = cross_val_score(pipeline, X3over_under, y3over_under, scoring='roc_auc', cv=5, n_jobs=-1)
    score = mean(scores)
    print('SMOTE oversampling rate:%.1f, Random undersampling rate:%.1f , Mean ROC AUC: %.3f' % (o, u, score))


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/imblearn/pipeline.py", line 262, in fit
    Xt, yt = self._fit(X, y, **fit_params_steps)
  File "/usr/local/lib/python3.7/dist-packages/imblearn/pipeline.py", line 226, in _fit
    **fit_params_steps[name],
  File "/usr/local/lib/python3.7/dist-packages/joblib/memory.py", l

SMOTE oversampling rate:0.3, Random undersampling rate:0.7 , Mean ROC AUC: nan
SMOTE oversampling rate:0.3, Random undersampling rate:0.6 , Mean ROC AUC: nan
SMOTE oversampling rate:0.3, Random undersampling rate:0.5 , Mean ROC AUC: nan
SMOTE oversampling rate:0.4, Random undersampling rate:0.7 , Mean ROC AUC: nan


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/imblearn/pipeline.py", line 262, in fit
    Xt, yt = self._fit(X, y, **fit_params_steps)
  File "/usr/local/lib/python3.7/dist-packages/imblearn/pipeline.py", line 226, in _fit
    **fit_params_steps[name],
  File "/usr/local/lib/python3.7/dist-packages/joblib/memory.py", l

SMOTE oversampling rate:0.4, Random undersampling rate:0.6 , Mean ROC AUC: nan
SMOTE oversampling rate:0.4, Random undersampling rate:0.5 , Mean ROC AUC: nan
SMOTE oversampling rate:0.5, Random undersampling rate:0.7 , Mean ROC AUC: nan
SMOTE oversampling rate:0.5, Random undersampling rate:0.6 , Mean ROC AUC: nan
SMOTE oversampling rate:0.5, Random undersampling rate:0.5 , Mean ROC AUC: nan


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 5.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/imblearn/pipeline.py", line 262, in fit
    Xt, yt = self._fit(X, y, **fit_params_steps)
  File "/usr/local/lib/python3.7/dist-packages/imblearn/pipeline.py", line 226, in _fit
    **fit_params_steps[name],
  File "/usr/local/lib/python3.7/dist-packages/joblib/memory.py", l